In [1]:
!pwd

/Users/nakatani/MetaLearning/hw1


In [3]:
import numpy as np
import os
import random
import tensorflow as tf
from scipy import misc
import warnings
from load_data import DataGenerator
from tensorflow.python.platform import flags
from tensorflow.keras import layers

warnings.simplefilter('ignore')

In [127]:
def get_images(paths, labels, nb_samples=None, shuffle=True):
    """
    Takes a set of character folders and labels and returns paths to image files
    paired with labels.
    Args:
        paths: A list of character folders
        labels: List or numpy array of same length as paths
        nb_samples: Number of images to retrieve per character
    Returns:
        List of (label, image_path) tuples
    """
    if nb_samples is not None:
        sampler = lambda x: random.sample(x, nb_samples)
    else:
        sampler = lambda x: x
    images_labels = [(i, os.path.join(path, image))
                     for i, path in zip(labels, paths)
                     for image in sampler(os.listdir(path))]
    if shuffle:
        random.shuffle(images_labels)
    return images_labels

In [128]:
def image_file_to_array(filename, dim_input):
    """
    Takes an image path and returns numpy array
    Args:
        filename: Image filename
        dim_input: Flattened shape of image
    Returns:
        1 channel image
    """
    image = misc.imread(filename)
    image = image.reshape([dim_input])
    image = image.astype(np.float32) / 255.0
    image = 1.0 - image
    return image

In [129]:
class DataGenerator(object):
    """
    Data Generator capable of generating batches of Omniglot data.
    A "class" is considered a class of omniglot digits.
    """

    def __init__(self, num_classes, num_samples_per_class, config={}):
        """
        Args:
            num_classes: Number of classes for classification (K-way)
            num_samples_per_class: num samples to generate per class in one batch
            batch_size: size of meta batch size (e.g. number of functions)
        """
        self.num_samples_per_class = num_samples_per_class
        self.num_classes = num_classes

        data_folder = config.get('data_folder', './omniglot_resized')
        self.img_size = config.get('img_size', (28, 28))

        self.dim_input = np.prod(self.img_size)
        self.dim_output = self.num_classes
        
        #  this contains all character-paths
        character_folders = [os.path.join(data_folder, family, character)
                             for family in os.listdir(data_folder)
                             if os.path.isdir(os.path.join(data_folder, family))
                             for character in os.listdir(os.path.join(data_folder, family))
                             if os.path.isdir(os.path.join(data_folder, family, character))]
#         print(character_folders)

        random.seed(1)
        random.shuffle(character_folders)
        num_val = 100
        num_train = 1100
        self.metatrain_character_folders = character_folders[: num_train]
        self.metaval_character_folders = character_folders[
            num_train:num_train + num_val]
        self.metatest_character_folders = character_folders[
            num_train + num_val:]

    def sample_batch(self, batch_type, batch_size):
        """
        Samples a batch for training, validation, or testing
        Args:
            batch_type: train/val/test
        Returns:
            A a tuple of (1) Image batch and (2) Label batch where
            image batch has shape [B, K, N, 784] and label batch has shape [B, K, N, N]
            where B is batch size, K is number of samples per class, N is number of classes
        """
        if batch_type == "train":
            folders = self.metatrain_character_folders
        elif batch_type == "val":
            folders = self.metaval_character_folders
        else:
            folders = self.metatest_character_folders

        #############################
        #### YOUR CODE GOES HERE ####
        
        B = batch_size
        K = self.num_samples_per_class
        N = self.num_classes
        
        all_image_batches = []
        all_label_batches= []
        
        for b in range(B):
            paths = random.sample(folders, N)
            labels = paths + ""
            images_labels = get_images(paths, labels, nb_samples=K, shuffle=False) # List of (label, image_path) tuples
            ib3 = []
            lb3 = []
            for k in range(K):
                i2 = []
                lb2 = []
                for n in range(N):
                    tpl = img_lables[n*K+k]
                    label ,filename = tpl                
                    img_array = image_file_to_array(filename, 784)
                    label_onehot = np.zeros(N - 1)
                    if n == N-1: label_onehot.insert(-1,1)
                    else: label_onehot.insert(n,1)
                    ib2.append(img_array) # creating N * 784
                    lb2.append(label_onehot)
                ib3.append(ib2) # creating K * N * 784
                lb3.append(lb2)
            all_image_batches.append(ib3) # creating B * K * N * 784
            all_label_batches.append(lb3) # creating B * K * N * N
                
                
        #############################

        return all_image_batches, all_label_batches


In [135]:
FLAGS = flags.FLAGS

flags.DEFINE_integer(
    'num_classes', 5, 'number of classes used in classification (e.g. 5-way classification).')

flags.DEFINE_integer('num_samples', 1,
                     'number of examples used for inner gradient update (K for K-shot learning).')

flags.DEFINE_integer('meta_batch_size', 16,
                     'Number of N-way classification tasks per batch')

DuplicateFlagError: The flag 'num_classes' is defined twice. First from /Users/nakatani/.pyenv/versions/anaconda3-5.3.0/envs/meta/lib/python3.7/site-packages/ipykernel_launcher.py, Second from /Users/nakatani/.pyenv/versions/anaconda3-5.3.0/envs/meta/lib/python3.7/site-packages/ipykernel_launcher.py.  Description from first occurrence: number of classes used in classification (e.g. 5-way classification).

In [131]:
def loss_function(preds, labels):
    """
    Computes MANN loss
    Args:
        preds: [B, K+1, N, N] network output
        labels: [B, K+1, N, N] labels
    Returns:
        scalar loss
    """
    #############################
    #### YOUR CODE GOES HERE ####
    pass
    #############################

In [132]:
class MANN(tf.keras.Model):

    def __init__(self, num_classes, samples_per_class):
        super(MANN, self).__init__()
        self.num_classes = num_classes
        self.samples_per_class = samples_per_class
        self.layer1 = tf.keras.layers.LSTM(128, return_sequences=True)
        self.layer2 = tf.keras.layers.LSTM(num_classes, return_sequences=True)

    def call(self, input_images, input_labels):
        """
        MANN
        Args:
            input_images: [B, K+1, N, 784] flattened images
            labels: [B, K+1, N, N] ground truth labels
        Returns:
            [B, K+1, N, N] predictions
        """
        #############################
        #### YOUR CODE GOES HERE ####
        pass
        #############################
        return out

In [133]:
ims = tf.placeholder(tf.float32, shape=(
    None, FLAGS.num_samples + 1, FLAGS.num_classes, 784))
labels = tf.placeholder(tf.float32, shape=(
    None, FLAGS.num_samples + 1, FLAGS.num_classes, FLAGS.num_classes))

data_generator = DataGenerator(
    FLAGS.num_classes, FLAGS.num_samples + 1)

o = MANN(FLAGS.num_classes, FLAGS.num_samples + 1)
out = o(ims, labels)

loss = loss_function(out, labels)
optim = tf.train.AdamOptimizer(0.001)
optimizer_step = optim.minimize(loss)

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())

    for step in range(50000):
        i, l = data_generator.sample_batch('train', FLAGS.meta_batch_size)
        feed = {ims: i.astype(np.float32), labels: l.astype(np.float32)}
        _, ls = sess.run([optimizer_step, loss], feed)

        if step % 100 == 0:
            print("*" * 5 + "Iter " + str(step) + "*" * 5)
            i, l = data_generator.sample_batch('test', 100)
            feed = {ims: i.astype(np.float32),
                    labels: l.astype(np.float32)}
            pred, tls = sess.run([out, loss], feed)
            print("Train Loss:", ls, "Test Loss:", tls)
            pred = pred.reshape(
                -1, FLAGS.num_samples + 1,
                FLAGS.num_classes, FLAGS.num_classes)
            pred = pred[:, -1, :, :].argmax(2)
            l = l[:, -1, :, :].argmax(2)
            print("Test Accuracy", (1.0 * (pred == l)).mean())


UnrecognizedFlagError: Unknown command line flag 'f'

In [106]:
a = [i for i in range(5)]
a

[0, 1, 2, 3, 4]

In [107]:
img = np.array([a for i in range(3)])
img

array([[0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4]])

In [108]:
b.shape

(3, 5)

In [109]:
imgs = np.array([[[i+1 for i in range(5)] for j in range(3)] for k in range(4)])
imgs

array([[[1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5]],

       [[1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5]],

       [[1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5]],

       [[1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5]]])

In [110]:
imgs.shape
# 5 by 3 img * 4

(4, 3, 5)

In [111]:
B = 2
c = np.array([[b for i in range(3)] for j in range(B)])


In [112]:
c.shape

(2, 3, 3, 5)

In [113]:
d = (random.sample(list(c), 2))
d

[array([[[0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4]],
 
        [[0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4]],
 
        [[0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4]]]),
 array([[[0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4]],
 
        [[0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4]],
 
        [[0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4]]])]

In [114]:
np.array(d).shape

(2, 3, 3, 5)

In [115]:
character_folders

NameError: name 'character_folders' is not defined

In [116]:
data_folder

NameError: name 'data_folder' is not defined

In [117]:
data_generator = DataGenerator(
    5, 1 + 1)

In [118]:
data_generator.character_folders

AttributeError: 'DataGenerator' object has no attribute 'character_folders'

In [119]:
# def image_file_to_array(filename, dim_input):


image_file_to_array(filename, 784)

NameError: name 'filename' is not defined

In [120]:
f = (1,2)
f1, f2  = f

In [ ]:
get_images(paths, labels, nb_samples=None, shuffle=True)

In [87]:
f1

1

In [88]:
f2

2

In [95]:
ii = [1]
mm = [2]
listtt = []
listtt.append(ii)
print(listtt)
listtt.append(mm)
print(listtt)
np.array(listtt).shape

[[1]]
[[1], [2]]


(2, 1)

In [97]:
im = [(i, j) for i in range(5) for j in range(4)]

In [98]:
im

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3)]

In [30]:
!python hw1.py --num_classes=2 --num_samples=1 --meta_batch_size=4

/Users/nakatani/.pyenv/versions/anaconda3-5.3.0/envs/meta/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/nakatani/.pyenv/versions/anaconda3-5.3.0/envs/meta/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/nakatani/.pyenv/versions/anaconda3-5.3.0/envs/meta/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("q

*****Iter 1800*****
Train Loss: 0.6931472 Test Loss: 0.69313246
Test Accuracy 0.495
*****Iter 1900*****
Train Loss: 0.6931472 Test Loss: 0.69314873
Test Accuracy 0.49
*****Iter 2000*****
Train Loss: 0.6931472 Test Loss: 0.69315237
Test Accuracy 0.485
*****Iter 2100*****
Train Loss: 0.6931472 Test Loss: 0.6931436
Test Accuracy 0.51
*****Iter 2200*****
Train Loss: 0.6931472 Test Loss: 0.69314766
Test Accuracy 0.495
*****Iter 2300*****
Train Loss: 0.6931472 Test Loss: 0.69314605
Test Accuracy 0.5
*****Iter 2400*****
Train Loss: 0.6931378 Test Loss: 0.693154
Test Accuracy 0.495
*****Iter 2500*****
Train Loss: 0.6931472 Test Loss: 0.693045
Test Accuracy 0.485
*****Iter 2600*****
Train Loss: 0.6931472 Test Loss: 0.6931466
Test Accuracy 0.505
*****Iter 2700*****
Train Loss: 0.6931472 Test Loss: 0.6931474
Test Accuracy 0.5
*****Iter 2800*****
Train Loss: 0.6931472 Test Loss: 0.6931441
Test Accuracy 0.49
*****Iter 2900*****
Train Loss: 0.6931472 Test Loss: 0.69315606
Test Accuracy 0.49
*****Ite

In [26]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

In [13]:
!conda install pillow -y

Solving environment: done

## Package Plan ##

  environment location: /Users/nakatani/.pyenv/versions/anaconda3-5.3.0/envs/meta

  added / updated specs:
    - pillow


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    freetype-2.10.4            |       ha233b18_0         891 KB
    lcms2-2.11                 |       h92f6f08_0         385 KB
    libtiff-4.1.0              |       hcb84e12_1         512 KB
    lz4-c-1.9.2                |       h79c402e_3         152 KB
    pillow-8.0.0               |   py37h1a82f1a_0         648 KB
    zstd-1.4.5                 |       h41d2c2f_0         1.0 MB
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

  freetype           pkgs/main/osx-64::freetype-2.10.4-ha233b18_0
  lcms2              pkgs/main/osx-64::lcms2-2.11-h9

In [11]:
sp.__version__

'1.1.0'

In [10]:
import scipy as sp

In [17]:
a = [0]*3

In [18]:
a

[0, 0, 0]

In [29]:
!conda install scikit-learn -y

Solving environment: done

## Package Plan ##

  environment location: /Users/nakatani/.pyenv/versions/anaconda3-5.3.0/envs/meta

  added / updated specs:
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    joblib-0.17.0              |             py_0         205 KB
    llvm-openmp-10.0.0         |       h28b9765_0         270 KB
    scikit-learn-0.23.2        |   py37h959d312_0         6.3 MB
    threadpoolctl-2.1.0        |     pyh5ca1d4c_0          16 KB
    ------------------------------------------------------------
                                           Total:         6.8 MB

The following NEW packages will be INSTALLED:

  joblib             pkgs/main/noarch::joblib-0.17.0-py_0
  llvm-openmp        pkgs/main/osx-64::llvm-openmp-10.0.0-h28b9765_0
  scikit-learn       pkgs/main/osx-64::scikit-learn-0.23.2-py37h959d312_0
  threadpoolctl      pkgs/main/noarch::th